In [1]:
import os
os.chdir('../../')

In [2]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import random

from main import parse_args_and_config, Diffusion
from datasets import inverse_data_transform

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.benchmark = False

###############################################################################
# 1) Notebook에서 sys.argv를 직접 설정 (argparse 흉내)
###############################################################################
sys.argv = [
    "main.py",
    "--config", "imagenet64.yml",  # 사용하려는 config
    "--sample",
    "--sample_type", "lagrangesolver",
    "--dpm_solver_type", "data_prediction",
    "--dpm_solver_order", "1",
    "--timesteps", "1000",
    "--skip_type", "logSNR",
    "--ni"
]

###############################################################################
# 2) 인자/설정 로드
###############################################################################
args, config = parse_args_and_config()

###############################################################################
# 3) Diffusion 객체 생성 -> 모델 로딩
###############################################################################
diffusion = Diffusion(args, config, rank=0)
diffusion.prepare_model()
diffusion.model.eval()

###############################################################################
# 4) 배치(25장) 한 번에 샘플링 -> 5x5 그리드(여백 없이) 시각화
###############################################################################
device = diffusion.device

INFO - main.py - 2025-03-30 07:11:50,581 - Using device: cuda


[prepare_model] Model is ready.


In [3]:
from tqdm import tqdm

def sample_in_batches(diffusion, config, device, total_samples=1024, batch_size=16):
    pairs = []
    for i in tqdm(range(0, total_samples, batch_size)):
        bs = min(batch_size, total_samples - i)
        noise = np.random.randn(bs, config.data.channels, config.data.image_size, config.data.image_size).astype(np.float32)
        noise = torch.tensor(noise, device=device)
        data, _ = diffusion.sample_image(noise, diffusion.model)
        pair = torch.stack([noise, data], dim=1)
        pairs.append(pair)
    return torch.cat(pairs, dim=0)

total_samples = 256
pairs = sample_in_batches(diffusion, config, device, total_samples=total_samples, batch_size=16)
print(pairs.shape)

100%|██████████| 16/16 [21:32<00:00, 80.77s/it]

torch.Size([256, 2, 3, 64, 64])


In [6]:
!mkdir -p /data/optimization/
save_file = f'/data/optimization/euler_NFE=1000_N={total_samples}_imagenet64.pt'
torch.save(pairs.data.cpu(), save_file)
pairs_load = torch.load(save_file)
pairs_load.shape

torch.Size([256, 2, 3, 64, 64])

In [5]:
print('done')

done
